In [1]:
import numpy as np
from scipy.io import loadmat
import open3d
import scipy
from scipy.spatial import distance
from open3d import geometry

def affine_transform(pc, R, t):
    return np.array([R.dot(a)+t for a in pc])
    
def background_removal(a_1, n_1):
    valid_int = (a_1[:,2])<1
    a_1 = a_1[valid_int]
    n_1 = n_1[valid_int]
    
    return a_1, n_1

def remove_nan(points, normals):
    keep = []
    for nan in np.isnan(normals):
        if (nan == True).any():
            keep.append(False)
        else:
            keep.append(True)
    points = points[keep]
    normals = normals[keep]
    
    return points, normals

def rigid_motion(p,q):
    """
    Least-Squares Rigid Motion Using Singular Value Decomposition. 
    (https://igl.ethz.ch/projects/ARAP/svd_rot.pdf) 
    
    (note: so far only for the easy case, where all weights are = 1)
    
    p,q: shape [num_points, 3]
    
    """
    n,d = p.shape
    
    # compute centroids
    p_cen = sum(p)/len(p)
    q_cen = sum(q)/len(q)
    
    # compute centered vectors
    X = np.array([i-p_cen for i in p])
    Y = np.array([i-q_cen for i in q])
    
    # compute covariance matrix 
    W = np.eye(n)
    S =  X.T.dot(W).dot(Y)
    
    # compute sigular value decomposition
    U, _, V = np.linalg.svd(S)
    
    # compute optimal R and t
    M = np.eye(d)
    M[-1,-1] = np.linalg.det(V.T.dot(U.T))
    R = V.T.dot(M).dot(U.T)
    
    t = q_cen - R.dot(p_cen)
    
    return R, t

def rms_error(p, q):
    n = p.shape[0]
    dist = [distance.euclidean(p[i,:], q[i,:]) for i in range(n)]
    return np.sqrt(np.sum(np.power(dist, 2))/n)

def show_fitting_result(pcds_list):
    
    point_clouds_object_list = []
    pc = open3d.PointCloud()
    for i, pcd in enumerate(pcds_list):
        point_clouds_object_list.append(open3d.PointCloud())
        point_clouds_object_list[i].points = open3d.Vector3dVector(pcd)
    
    open3d.draw_geometries(point_clouds_object_list)

    
def informative_subsampling(normals, sample_size):
    # convert normals to angular space
    b = np.sqrt(normals[:,0]**2+normals[:,1]**2)
    x = np.arctan2(normals[:,1], normals[:,0])
    y = np.arctan2(normals[:,2], b)
    
    # devide normals over bins
    bins = np.linspace(-np.pi, np.pi, sample_size) 
    x_index = np.digitize(x, bins, right=True)
    y_index = np.digitize(y, bins, right=True)
    index = x_index * sample_size + y_index

    # uniformly sample from bins
    unique_index, original_index = np.unique(index, return_index=True)
    samples = np.random.choice(unique_index.shape[0], sample_size, replace=False)
    sample_index = original_index[samples]
    
    # return only the found sample indices of the original pointcloud
    return sample_index
   
def icp(a_1, a_2, n_1, n_2, convergence_treshold=0.005, point_selection="all", sample_size=3000, generate_3d = True, verbose = True, is_test = False , accuracy_check = False, stability_constant = 1):
    """
    a_1: positions of points in point cloud 1. shape : [num_points1, 3]
    a_2: positions of points in point cloud 2. shape : [num_points2, 3]
    """
    
    if is_test:
        generate_3d = False
        verbose = False
    n,d = a_1.shape
    
    
    # Filter the point clouds based on the depth,
    # only keep the indices where the z of the point cloud is less than 1
    a_1, n_1 = background_removal(a_1, n_1)
    a_2, n_2 = background_removal(a_2, n_2)
    
    a_2_c = a_2.copy()
    # Point selection
    # Uniform subsampling
    if point_selection == "uniform":
        a_1 = a_1[np.random.randint(low=0, high=a_1.shape[0], size=sample_size)]
        a_2 = a_2[np.random.randint(low=0, high=a_2.shape[0], size=sample_size)]
        
    if point_selection == "informative":
        a_1 = a_1[informative_subsampling(n_1, sample_size), :]
        a_2 = a_2[informative_subsampling(n_2, sample_size), :]
    
    if stability_constant == 1 :
        R_overall = np.eye(d)
        t_overall = np.zeros(d)
    else:
        R_overall = np.random.normal(0,1, size = (d,d))*stability_constant
        t_overall = np.random.normal(0,1, size = d)*stability_constant
    
    # Base loop on difference in rsm error
    rms_error_old = 10000
    rms_error_new = rms_error_old-1
    
    while rms_error_old-rms_error_new > convergence_treshold:
        
        if point_selection == "random":
            a_1 = a_1[np.random.choice(a_1.shape[0], sample_size, replace=False), :]
            a_2 = a_2[np.random.choice(a_2.shape[0], sample_size, replace=False), :]
            
        # (Step 1) Find closest points for each point in a_1 from a_2
        tree = scipy.spatial.KDTree(a_2)
        closest_dists, closest_idx = tree.query(a_1)
        # Found this on stackoverflow: https://bit.ly/2P8IYiw
        # Not sure if we can use this, but it is definetely much (!!) faster 
        # than manually comparing all the vectors.
        # Usage also proposed on Wikipedia: https://bit.ly/2urg9nU
        # For how-to-use see: https://bit.ly/2UbKNfn
        closest_a_2 = a_2[closest_idx]
    
        # (Step 2) Refine R and t using Singular Value Decomposition
        R, t = rigid_motion(a_1,closest_a_2)
       
        # update a_1
        a_1 =affine_transform(a_1, R, t)
        
        # update rms error
        rms_error_old = rms_error_new
        rms_error_new = rms_error(a_1, closest_a_2)
        
        if verbose:
            print(rms_error_new)
        
        # update overall R and t
        R_overall = R.dot(R_overall)
        t_overall = R.dot(t_overall) + t
    if generate_3d:
        show_fitting_result([a_1, a_2_c])
    if accuracy_check:
        return rms_error_new
    return R_overall, t_overall
    

In [2]:
# print("Load a ply point cloud, print it, and render it")
# pcd1  = open3d.read_point_cloud("Data/data/0000000000.pcd")
# pcd2  = open3d.read_point_cloud("Data/data/0000000001.pcd")
# print(np.asarray(pcd.points))
# open3d.draw_geometries([pcd])
#a_1 = loadmat("Data/source.mat")["source"].T
#a_2 = loadmat("Data/target.mat")["target"].T

a_1 = open3d.read_point_cloud("Data/data/0000000000.pcd")
a_2 = open3d.read_point_cloud("Data/data/0000000001.pcd")

a_1 = np.asarray(a_1.points)
a_2 = np.asarray(a_2.points)

n_1 = open3d.read_point_cloud("Data/data/0000000000_normal.pcd", format='xyz')
n_2 = open3d.read_point_cloud("Data/data/0000000001_normal.pcd", format='xyz')

n_1 = np.asarray(n_1.points)
n_2 = np.asarray(n_2.points)

a_1, n_1 = remove_nan(a_1, n_1)
a_2, n_2 = remove_nan(a_2, n_2)

In [3]:
icp(a_1=a_1, a_2=a_2, n_1=n_1, n_2=n_2, point_selection = 'informative', is_test=True)

(array([[ 0.99926009, -0.00345711, -0.0383055 ],
        [ 0.00334233,  0.99998973, -0.00306018],
        [ 0.03831569,  0.00292988,  0.99926139]]),
 array([ 0.03411603,  0.00242297, -0.00066212]))

In [ ]:
sampling_methods = ['random', 'uniform', 'default', 'informative']

In [ ]:
def speed_check(a_1, a_2, n_1, n_2, sampling_methods):
    for method in sampling_methods:
        print("Time that it takes for {} method is:".format(method))
        %time  icp(a_1, a_2, n_1, n_2, point_selection = method, is_test = True)
speed_check(a_1, a_2, n_1, n_2, sampling_methods)

Time that it takes for random method is:
CPU times: user 1.61 s, sys: 28 ms, total: 1.64 s
Wall time: 1.32 s
Time that it takes for uniform method is:
CPU times: user 1.65 s, sys: 61.7 ms, total: 1.71 s
Wall time: 1.3 s
Time that it takes for default method is:


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Users/mbonnee/miniconda3/lib/python3.7/site-packages/IPython/core/magics/execution.py", line 1284, in time
    out = eval(code, glob, local_ns)
  File "<timed eval>", line 1, in <module>
  File "<ipython-input-1-de8f3f00378c>", line 158, in icp
    R, t = rigid_motion(a_1,closest_a_2)
  File "<ipython-input-1-de8f3f00378c>", line 52, in rigid_motion
    S =  X.T.dot(W).dot(Y)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/mbonnee/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 2033, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/mbonnee/miniconda3/lib/python3.7/site-packages/IPython/core/ultratb.py", line 1095, in get_

KeyboardInterrupt: 

Time that it takes for informative method is:
CPU times: user 1.94 s, sys: 126 ms, total: 2.06 s
Wall time: 1.69 s


In [ ]:
def accuracy_check(a_1, a_2, n_1, n_2, sampling_methods):
    for method in sampling_methods:
        print("RMSE of {} method is {:.4}".format(method, icp(a_1, a_2, n_1, n_2, point_selection = method, is_test = True, accuracy_check = True)))
accuracy_check(a_1, a_2, n_1, n_2, sampling_methods)
    

RMSE of random method is 0.005258
RMSE of uniform method is 0.00529


In [ ]:
def  noise_check(a_1, a_2, n_1, n_2, sampling_methods):
    noise_1, noise_2 = np.random.normal(0,1,(a_1.shape)) ,np.random.normal(0,1,(a_2.shape))
    a_1_noisy, a_2_noisy = a_1 + noise_1 , a_2 + noise_2
    for method in sampling_methods:
        rmse_normal =icp(a_1, a_2, n_1, n_2, point_selection = method, is_test = True, accuracy_check = True) 
        rmse_noisy = icp(a_1_noisy, a_2_noisy, n_1, n_2, point_selection = method, is_test = True, accuracy_check = True)
        print("RMSE of {} method is {:.4}, whereas if we add noise, it becomes {:.4} , the difference is {:.4}".format(method, rmse_normal, rmse_noisy, rmse_normal - rmse_noisy))
        R_normal , t_normal = icp(a_1, a_2, n_1, n_2, point_selection = method, is_test = True) 
        R_noisy , t_noisy = icp(a_1_noisy, a_2_noisy, n_1, n_2, point_selection = method, is_test = True) 
        R_distance, t_distance = np.linalg.norm(R_normal - R_noisy), np.linalg.norm(t_normal - t_noisy)
        print("The distance between normal and noisy R matrices is {:.4} and between normal and noisy t vectors is {:.4}".format(R_distance, t_distance))
        

noise_check(a_1, a_2, n_1, n_2, sampling_methods)
    

In [ ]:
def  stability_check(a_1, a_2, sampling_methods):
    stability_constant = 10
    for method in sampling_methods:
        rmse_normal =icp(a_1, a_2, n_1, n_2, point_selection = method, is_test = True, accuracy_check = True) 
        rmse_non_stable= icp(a_1, a_2, n_1, n_2, point_selection = method, is_test = True, accuracy_check = True, stability_constant = stability_constant)
        print("RMSE of {} method is {:.4}, whereas if we have random initialisation, it becomes {:.4} , the difference is {:.4}".format(method, rmse_normal, rmse_non_stable, rmse_normal - rmse_non_stable))
        R_normal , t_normal = icp(a_1, a_2, n_1, n_2, point_selection = method, is_test = True) 
        R_non_stable , t_non_stable = icp(a_1, a_2, n_1, n_2, point_selection = method, is_test = True, stability_constant = stability_constant) 
        R_distance, t_distance = np.linalg.norm(R_normal - R_non_stable), np.linalg.norm(t_normal - t_non_stable)
        print("The distance between normal and randomely initialised R matrices is {:.4} and between normal and randomely initialised t vectors is {:.4}".format(R_distance, t_distance))
        

stability_check(a_1, a_2, n_1, n_2, sampling_methods)
    

Note: Take stability as an example, an experimental setup may be the following: given a source point cloud, augmented target point cloud can be obtained by transforming the source point cloud using a random rigid transform R and t. Stability can be analyzed by observing a behaviour of ICP on source and augmented target point cloud w.r.t changes in magnitude of R and t.
        

- stability is about convergence of the algorithm dependent on initial condition.

- tolerance to noise is about convergence of the algorithm with input data with noise. You can imagine data is captured by a sensor. In the ideal case you will obtain exact point cloud, however sensor is not precise, therefore there will be noise in measurement. Therefore we ask you to evaluate how ICP is robust against those kind of issuses.

In [ ]:
import os
import re

def read_pcds_from_filenames(filenames):
    """
    Read point clouds for given file names.
    """
    return [open3d.read_point_cloud(f)for f in filenames]

def affine_transform(pc, R, t):
    return np.array([R.dot(a)+t for a in pc])
    
def background_removal(a_1):
    valid_bool_1 = (a_1[:,2])<1
    a_1 = a_1[valid_bool_1]
    return a_1
    
    
## Exercise 3.1
def merging_scenes(frame_interval=1, point_selection='random', data_dir="./Data/data/"):
        
    # Read all filenames for given directory
    filenames = [data_dir+x for x in os.listdir(data_dir) if re.match(r"00000000[0-9][0-9].pcd",x)]
    filenames.sort()
    
    # Get relevant filenames (according to frame_interval)
    filenames=filenames[0::frame_interval]
    
    # Get point clouds for relevant filnames
    pcds = read_pcds_from_filenames(filenames)
    pcd_points = [np.array(p.points) for p in pcds]
    
    
    # Tansform all frames back to zero-frame space 
    R_to_zero_current = np.eye(3)
    t_to_zero_current = np.zeros(3)
    transformed_points = []
    
    for i in range(1,len(pcds)):
        
        # Perform ICP
        R,t = icp(pcd_points[i-1], pcd_points[i], is_test=True, point_selection=point_selection)
        
        # Update transformations back to zero frame
        R_to_zero_current = R.dot(R_to_zero_current)
        t_to_zero_current = R.dot(t_to_zero_current) + t
        
        # Project current PointCloud back to 0-frame space
        transformed_points += [affine_transform(pcd_points[i],R_to_zero_current,t_to_zero_current)]
    
    # Remove backgrounds in our pcds
    # pcd_points = [background_removal(p) for p in pcd_points]
    
    
    # Apply transformation to pcd
    # pcd_points_transformed = [affine_transform(pcd_points[i],R_list[i], t_list[i]) 
                              #for i in range(len(pcd_points)-1)]
    
    #for i in range(len(pcds)):
    #    pcds[i].points = pcd_points_transformed[i]
        
    show_fitting_result(transformed_points)
    
    
merging_scenes(frame_interval=1)

In [ ]:
import os
import re

def read_pcds_from_filenames(filenames):
    """
    Read point clouds for given file names.
    """
    return [open3d.read_point_cloud(f)for f in filenames]

def affine_transform(pc, R, t):
    return np.array([R.dot(a)+t for a in pc])
    
def background_removal(a_1):
    valid_bool_1 = (a_1[:,2])<1
    a_1 = a_1[valid_bool_1]
    return a_1
    
    
## Exercise 3.1
def merging_scenes(frame_interval=1, point_selection='random', data_dir="./Data/data/"):
        
    # Read all filenames for given directory
    filenames = [data_dir+x for x in os.listdir(data_dir) if re.match(r"00000000[0-9][0-9].pcd",x)]
    filenames.sort()
    
    # Get relevant filenames (according to frame_interval)
    filenames=filenames[0::frame_interval]
    
    # Get point clouds for relevant filnames
    pcds = read_pcds_from_filenames(filenames)
    pcd_points = [background_removal(np.array(p.points)) for p in pcds]
    
    
    # Tansform all frames back to zero-frame space 
    R_to_zero_current = np.eye(3)
    t_to_zero_current = np.zeros(3)
    transformed_points = []
    
    for i in range(1, len(pcd_points)):
        
        # Perform ICP
        R,t = icp(pcd_points[i], pcd_points[i-1], is_test=True, point_selection=point_selection)
        
        # Update transformations back to zero frame
        R_to_zero_current = R.dot(R_to_zero_current)
        t_to_zero_current = R.dot(t_to_zero_current) + t
        
        # Project current PointCloud back to 0-frame space
        
        transformed_points += [affine_transform(pcd_points[i-1],R_to_zero_current,t_to_zero_current)]
    
    # Remove backgrounds in our pcds
    # pcd_points = [background_removal(p) for p in pcd_points]
    
    
    # Apply transformation to pcd
    # pcd_points_transformed = [affine_transform(pcd_points[i],R_list[i], t_list[i]) 
                              #for i in range(len(pcd_points)-1)]
    
    #for i in range(len(pcds)):
    #    pcds[i].points = pcd_points_transformed[i]
        
    show_fitting_result(transformed_points)
    
    
merging_scenes(frame_interval=1)